In [76]:
# Imported all of the modules I would need to utilize.  I commented out Plotly, since it is not available in my base environment, which is what I always use except for when creating Plotly visualizations.

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import plotly
# import plotly.express as px
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer

# This setting widens how many characters Pandas will display in a column:
pd.options.display.max_colwidth = 400

# Obtained spanish stop words list from: https://github.com/stopwords-iso/stopwords-es/blob/master/stopwords-es.txt
# Obtained guidance on creating stop_words_list using .txt file from: https://stackoverflow.com/questions/36369870/sklearn-how-to-add-custom-stopword-list-from-txt-file#36371942
with open('../data/spanish.txt', 'r') as file:
    spanish=[file.read().replace('\n', ',')]

## Data Cleaning

In [2]:
# Created a DataFrame from the .csv file I saved with Partido Acción Nacional's pulled tweets.
pan_df = pd.read_csv('../data/pan.csv')

In [3]:
# Created a DataFrame from the .csv file I saved with Partido Revolucionario Institucional's pulled tweets.
pri_df = pd.read_csv('../data/pri.csv')

In [4]:
# Created a DataFrame from the .csv file I saved with Partido de la Revolución Democrática's pulled tweets.
prd_df = pd.read_csv('../data/prd.csv')

In [5]:
# Created a DataFrame from the .csv file I saved with Partido Verde Ecologista de México's pulled tweets.
pv_df = pd.read_csv('../data/pv.csv')

In [6]:
# Created a DataFrame from the .csv file I saved with Movimiento Ciudadano's pulled tweets.
mc_df = pd.read_csv('../data/mc.csv')

In [7]:
# Created a DataFrame from the .csv file I saved with Morena's pulled tweets.
mrn_df = pd.read_csv('../data/mrn.csv')

In [8]:
# Concatenating Dataframes
mx_tweets = pd.concat([pan_df, pri_df, prd_df, pv_df, mc_df, mrn_df])

In [9]:
mx_tweets.shape

(19169, 3)

In [10]:
# Resetting the index of this concatenated DataFrame and ensuring shape didn't change.
mx_tweets.reset_index(inplace = True, drop = True)

In [11]:
mx_tweets.shape

(19169, 3)

In [12]:
mx_tweets.head()

,time,user,tweet
0,2022-06-06 22:52:00+00:00,AccionNacional,"¡Lo hicimos, Durango! El municipio sigue pintándose de azul y @josejoseantonio trabajará incansablemente por un gobierno cercano a la gente, que continúe dando resultados y mejore la calidad de vida de las familias👏🏼 https://t.co/XEYD1XdXQr"
1,2022-06-06 17:01:25+00:00,AccionNacional,"¡Ganamos Durango con @EVillegasV!\n\nGracias a las y los duranguenses por su confianza, trabajaremos juntos para que continúen los buenos resultados y el crecimiento en el estado.\n\n#EstebanGobernador https://t.co/j5HikIlkMv"
2,2022-06-06 15:01:41+00:00,AccionNacional,¡Muchas felicidades @TereJimenezE por tu triunfo contundente! Eres un orgullo para todo el panismo y estamos seguros te convertirás en la mejor gobernadora de Aguascalientes.\n\n#TereGobernadora https://t.co/5kuBaOMfWE
3,2022-06-06 14:06:27+00:00,AccionNacional,RT @TereJimenezE: ¡Hoy ganaron las propuestas y la unidad! ¡Hoy ganó Aguascalientes!\n\n¡GRACIAS! https://t.co/NzYv7dsqYb
4,2022-06-06 14:06:15+00:00,AccionNacional,"RT @EVillegasV: ¡Durango ganó con valor! Simplemente, ¡gracias a las y los duranguenses que me han otorgado su confianza! Hoy comienza el g…"


In [13]:
# Created new DataFrame named mx_political that contained only the feature column and the label.
mx_political = mx_tweets[['user', 'tweet']]

In [14]:
type(mx_political)

pandas.core.frame.DataFrame

In [15]:
mx_political.isna().sum()

user     0
tweet    0
dtype: int64

In [16]:
# Proceeded to save this new DataFrame (with only the feature and target columns) as a .csv file.
mx_political.to_csv('../data/mx_political.csv', index=False)

## Exploratory Data Analysis

In [17]:
#In order to perform Exploratory Data Analysis without altering the "mx_political" DataFrame, I created a copy of it and named it "mx_political_eda"
mx_political_eda = mx_political.copy()

In [18]:
mx_political_eda['user'].value_counts(normalize=True)

partidoverdemex    0.166936
PRI_Nacional       0.166884
AccionNacional     0.166832
PartidoMorenaMx    0.166780
PRDMexico          0.166519
MovCiudadanoMX     0.166049
Name: user, dtype: float64

In [19]:
mx_political_eda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19169 entries, 0 to 19168
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   user    19169 non-null  object
 1   tweet   19169 non-null  object
dtypes: object(2)
memory usage: 299.6+ KB


In [20]:
mx_political_eda.head(10)

,user,tweet
0,AccionNacional,"¡Lo hicimos, Durango! El municipio sigue pintándose de azul y @josejoseantonio trabajará incansablemente por un gobierno cercano a la gente, que continúe dando resultados y mejore la calidad de vida de las familias👏🏼 https://t.co/XEYD1XdXQr"
1,AccionNacional,"¡Ganamos Durango con @EVillegasV!\n\nGracias a las y los duranguenses por su confianza, trabajaremos juntos para que continúen los buenos resultados y el crecimiento en el estado.\n\n#EstebanGobernador https://t.co/j5HikIlkMv"
2,AccionNacional,¡Muchas felicidades @TereJimenezE por tu triunfo contundente! Eres un orgullo para todo el panismo y estamos seguros te convertirás en la mejor gobernadora de Aguascalientes.\n\n#TereGobernadora https://t.co/5kuBaOMfWE
3,AccionNacional,RT @TereJimenezE: ¡Hoy ganaron las propuestas y la unidad! ¡Hoy ganó Aguascalientes!\n\n¡GRACIAS! https://t.co/NzYv7dsqYb
4,AccionNacional,"RT @EVillegasV: ¡Durango ganó con valor! Simplemente, ¡gracias a las y los duranguenses que me han otorgado su confianza! Hoy comienza el g…"
5,AccionNacional,"RT @josejoseantonio: Con la reserva de los resultados oficiales, todas las encuestas de salida nos marcan como ganadores de la contienda. ¡…"
6,AccionNacional,Lamentamos profundamente el fallecimiento de nuestra diputada federal Alma Rosa Hernández Escobar. Nuestro más sentido pésame y un fuerte abrazo a sus seres queridos.\n\nDescanse en paz. https://t.co/PbfrM8fEJd
7,AccionNacional,"¡Ganamos contundentemente Aguascalientes y Durango, y vamos arriba en Tamaulipas!\n\nHoy queda claro que vamos en el camino correcto, seguimos avanzando y #SíHayTiro2024. https://t.co/lgDGvyVZ93"
8,AccionNacional,RT @JorgeRoHe: Hoy las y los ciudadanos dieron su confianza a los buenos gobiernos de @AccionNacional\n\n¡Vencimos el intento fallido de elec…
9,AccionNacional,"RT @MarkoCortes: Hoy quedó claro que a pesar de toda la ilegalidad y autoritarismo del Gobierno federal y Morena, avanzamos para en el 2024…"


In [21]:
mx_political_eda.tail(10)

,user,tweet
19159,PartidoMorenaMx,Impulsar una cultura de paz y eliminar todo tipo de violencia contras las mujeres es fundamental para construir una sociedad verdaderamente justa. Acompáñanos en las actividades en el marco del #DiaContralaViolenciadeGenero\n\nhttps://t.co/pTfHuv28AM
19160,PartidoMorenaMx,RT @DiputadosMorena: No más indiferencia ante los abusos
19161,PartidoMorenaMx,RT @DiputadosMorena: Vivas nos queremos #DiaContralaViolenciadeGenero
19162,PartidoMorenaMx,RT @DiputadosMorena: Ni una menos #NiUnaMenos
19163,PartidoMorenaMx,RT @DiputadosMorena: Exigimos respeto #DiaContralaViolenciadeGenero
19164,PartidoMorenaMx,RT @DiputadosMorena: Abajo el patriarcado #NiUnaMas
19165,PartidoMorenaMx,RT @DiputadosMorena: Hartas de violencias machistas #NiUnaMenos
19166,PartidoMorenaMx,"RT @DiputadosMorena: Vivas, libres y en lucha #DiaContralaViolenciadeGenero"
19167,PartidoMorenaMx,"RT @DiputadosMorena: No nos sentimos seguras, no nos sentimos a salvo #NiUnaMenos #25Nov"
19168,PartidoMorenaMx,RT @DiputadosMorena: Luchamos hoy para no morir mañana #DiaContralaViolenciadeGenero


In [22]:
# Created a new column named 'tweet_char_length' which contained the character length of each tweet.
mx_political_eda['tweet_char_length'] = mx_political_eda['tweet'].apply(len)

In [23]:
# Obtained white space counter function from: https://www.geeksforgeeks.org/python-program-to-count-the-number-of-spaces-in-string/
# Created a function named "check_space" that counted the number of spaces in each title and added 1 to obtain the total number of words.
def check_space(Test_string):
    return Test_string.count(" ") + 1

In [24]:
# Created a new column named 'title_word_count' which contained each title's word count.
mx_political_eda['tweet_word_count'] = mx_political_eda['tweet'].apply(check_space)

In [25]:
mx_political_eda.sort_values(by = 'tweet_word_count', ascending=False).head(10)

,user,tweet,tweet_char_length,tweet_word_count
13916,MovCiudadanoMX,¡Noticias chidas para las plebes! Es #8M.\n\n 💚 💚 💚 💚\n 💚 💚 💚\n 💚 Y Sinaloa 💚\n 💚 lo sabe. 💚\n 💚 💚\n 💚 💚 \n 💚,264,191
15456,MovCiudadanoMX,RT @BancadaNaranja: En todo México:\n\n 🌶 🌽 🥑 🧅\n 🥒 🫑 🍅\n 🫒 HAGAMOS 🍠…,135,84
15743,MovCiudadanoMX,RT @MovCiudadanoCMX: No les pasa que:\n 🏳️‍🌈 🏳️‍🌈 🏳️‍🌈 🏳️‍🌈\n 🏳️‍🌈 🏳️‍🌈 🏳️‍🌈\n 🏳️‍🌈…,140,73
11206,partidoverdemex,"@azcarmx @ernesto_mendez @WWF_LAC @unep_espanol @prensaanimal4 @lajornadaonline @ContraReplicaMX @heraldodemexico @ImagenTVMex @laoctavadigital @Reforma Buen día, igual que a ustedes, nos preocupan muchas cuestiones en torno a esta obra, una de ellas es el uso de diésel. Por ello, ya pedimos a @SCT_mx informe el impacto ambiental que genera su uso, al igual que priorizar fuentes sustentables q...",430,60
15133,MovCiudadanoMX,.@ClementeCH en #Tragaluz: 😎🖐\n 🎤\n\nhttps://t.co/70kz0mpxpW,109,57
15936,MovCiudadanoMX,"En el tercer informe de gobierno, el presidente olvidó informarnos sobre la falta de servicios de salud que tienen millones de personas y que hoy es un gran problema. 😢\n\nUrge un #NuevoTrato por la salud para que el acceso a ella sea un derecho y no un privilegio de unos cuantos. https://t.co/j2yssnO6ga",303,52
5234,PRI_Nacional,"Este 6 de junio, en #SalinaCruz y en #Oaxaca vamos a defender todo lo que hemos logrado, con un objetivo claro, asegurar la tranquilidad de las familias, el futuro de los jóvenes y la seguridad de las mujeres. \n\n¡No vamos a descansar ni un solo día por las y los mexicanos! https://t.co/ejcgj9umcW",297,52
292,AccionNacional,"En un encuentro con la Red por los Derechos de la Infancia en México, coincidimos que es fundamental cuidar a la niñez mexicana para que no sea carne de cañón de las mafias, para que tengan un presente y un futuro, donde su arranque sea más parejo y no haya desigualdades. https://t.co/XBUd9IWjaF",296,52
15670,MovCiudadanoMX,"@ZERIM78 Gracias por el voto de confianza, RAM; sin duda tenemos mucho por hacer en este Movimiento y lo haremos de la mano de las y los ciudadanos. El defender las causas ciudadanas, es la prioridad desde todos los espacios en los que nos encontramos. No les vamos a fallar. 🙌 😉",279,52
5750,PRI_Nacional,"Una de las tareas más importantes de los diputados es la de atender las necesidades de la gente, pero la mayoría de Morena en el Congreso sigue sin hacer nada para apoyar a los mexicanos frente a la crisis. Este 6 de junio vota por los que sí saben, ¡#VotaPRI! https://t.co/14OOwNJnom",284,51


In [27]:
mx_political_eda.sort_values(by = 'tweet_word_count', ascending=True).head(10)

,user,tweet,tweet_char_length,tweet_word_count
18059,PartidoMorenaMx,https://t.co/jr3x4eOrmX,23,1
16339,PartidoMorenaMx,https://t.co/WH9mcws1dx,23,1
15416,MovCiudadanoMX,🤦‍♀️\nhttps://t.co/DvRqk7HvOY,28,1
10093,partidoverdemex,https://t.co/QbU8uXaePr,23,1
16340,PartidoMorenaMx,https://t.co/ED6IWot6qL,23,1
6845,PRDMexico,*#SalvemosalIECM,16,1
17408,PartidoMorenaMx,https://t.co/Q9PaHYXigB,23,1
15177,MovCiudadanoMX,🤩👇\nhttps://t.co/iyo6EzYE0h,26,1
18044,PartidoMorenaMx,https://t.co/cKxXhga7mY,23,1
9930,partidoverdemex,@sdpnoticias\n@Siete24mx\n@ReporteIndigo\n@Excelsior\n@LaCronicaDeHoy\n@lajornadaonline\n@laprensaoem\n@Milenio\n@Reforma\n@SinEmbargoMX\n@Formato21GRC\n@Radio_Formula\n@MVSNoticias\n@Imagen_Mx\n@EfektoTv\n@NTelevisa_com\n@AztecaNoticias\n@ImagenTVMex\n@mileniotv\n@Pajaropolitico\n@EjeCentral,273,1


In [26]:
# Obtained the longest tweets by character length.
mx_political_eda.sort_values(by = 'tweet_char_length', ascending=False).head(10)

,user,tweet,tweet_char_length,tweet_word_count
11206,partidoverdemex,"@azcarmx @ernesto_mendez @WWF_LAC @unep_espanol @prensaanimal4 @lajornadaonline @ContraReplicaMX @heraldodemexico @ImagenTVMex @laoctavadigital @Reforma Buen día, igual que a ustedes, nos preocupan muchas cuestiones en torno a esta obra, una de ellas es el uso de diésel. Por ello, ya pedimos a @SCT_mx informe el impacto ambiental que genera su uso, al igual que priorizar fuentes sustentables q...",430,60
14978,MovCiudadanoMX,"@ClementeCH @VeroDelgadilloG @DanteDelgado @Chertorivski @Pat_MercadoC @AlvarezMaynez Para José Luis Rodríguez Zapatero la democracia tiene como características: la no discriminación, la redistribución, la protección de todos los derechos y el feminismo.\nAdemás, prioriza el diálogo, estando de lado de la ciencia y del conocimiento, nunca del fanatismo. https://t.co/LYxNAa8uC0",378,46
14975,MovCiudadanoMX,@ClementeCH @VeroDelgadilloG @IndiraKempis @Pat_MercadoC @Chertorivski @DanteDelgado @AlvarezMaynez “En #MovimientoCiudadano hemos representado una alternativa haciendo un llamado a todo el país para imaginarnos el México en el que queremos vivir: un México socialdemócrata. A ese llamado lo hemos denominado la #EvoluciónMexicana”: @VeroDelgadilloG. https://t.co/8pEDktkSLp,374,42
14972,MovCiudadanoMX,"@Pat_MercadoC @DanteDelgado @VeroDelgadilloG @IndiraKempis @Chertorivski @ClementeCH @AlvarezMaynez “La socialdemocracia es decir paz, persuasión, perseverancia, igualdad y libertad; sobre todo un querer vivir juntos”: José Luis Rodríguez Zapatero.\n\nEn #MovimientoCiudadano estamos convencidos de que el futuro es socialdemócrata, #ElFuturoEsNaranja. https://t.co/AdBxP3hQ9C",374,38
14239,MovCiudadanoMX,"@PaolaSaeb @SergioGilRu @Pablo_VAh En🇲🇽, más de 93,000 personas están detenidas sin sentencia por la prisión preventiva; la mayoría son pobres y no tienen una defensa adecuada, y son las mujeres las más afectadas.\nConoce más en #PalabrasEnMovimiento y analicemos juntas y juntos la realidad de la justicia en México",315,48
10615,partidoverdemex,"@Martha_Herrera @karencastrejon_ Hola Martha; seguimos luchando para erradicar el maltrato animal. Hemos promovido el uso de calesas eléctricas para demostrar su funcionalidad: https://t.co/S5iavv0T1g\nTu voz será de mucha ayuda al denunciar y sumar a la causa, en: https://t.co/qcMUs54XHN\nGracias por tu reporte",311,40
7465,PRDMexico,"@ZubeIb @PRI_Nacional @AccionNacional Si lo hemos hecho, el #PRD ha denunciado todos y cada uno de los atropellos del Presidente, funcionarios federales y miembros del partido Morena. Poco a poco se ha avanzado. Por ejemplo conseguimos un amparo para detener la desaparición de las #EscuelasDeTiempoCompleto. ☀️",311,46
18202,PartidoMorenaMx,"En lugar de Chipotle deberían ponerle Chayote... en vez de gastar más recursos en justificar sus privilegios, deberían destinarlos a garantizar la democracia para el pueblo.\n\nQuieren explicar #PerasConPeras y aquí un ejemplo de sus excesos con #PerasYManzanas 👇🏽 https://t.co/FHOe65elAi https://t.co/9qR8ORmS0R",310,40
9734,partidoverdemex,@MartaGuzmanOf @LauMondragon @verde_ags @verde_bcs @PVEMCAMPECHE @chihuahua_verde @VerdeCDMX @coahuila_verde @PVEMColima1 @PvemEstatalDgo @verdeedomex @partidoverdegto @verdeguerrero_ @partidoverdehgo @partidoverdejal @pvemmorelos @NayaritPve @PartidoVerde_NL @partidoverde_oa @PVEM_Puebla @PartidoVerdeQu1,306,21
16911,PartidoMorenaMx,"@CiroMurayamaINE @brozoxmiswebs No se equivoque, consejero, no nos “cala”, a diferencia de ustedes, la libertad de expresión. \n\nAl contrario, le sugerimos que manifieste sus posturas políticas desde un partido, transparentemente, y no desde un espacio donde su trabajo es garantizar imparcialidad. Saludos.",306,42


In [28]:
# Obtained the longest tweets by character length.
mx_political_eda.sort_values(by = 'tweet_char_length', ascending=True).head(10)

,user,tweet,tweet_char_length,tweet_word_count
13735,MovCiudadanoMX,🖼: @larenn16,12,2
15567,MovCiudadanoMX,@damarisgc 😳,12,2
14620,MovCiudadanoMX,@EriqORulo 🙁 💔,14,3
15495,MovCiudadanoMX,@ChavoNava 😎 🙌,14,3
14785,MovCiudadanoMX,@SophieYunes 😻,14,2
15669,MovCiudadanoMX,@GuroSilva 🧡 🙌,14,3
15529,MovCiudadanoMX,@DaffnyRoM 👍 😀,14,3
15913,MovCiudadanoMX,Cerramos hilo.,14,2
14579,MovCiudadanoMX,@ErikaHadaz 🙌 🍊,15,3
15479,MovCiudadanoMX,@gafer54gdl 🙌 ✨,15,3


### 10 Most Common Unigrams

In [77]:
# Instantiating a CountVectorizer object that splits the documents in the corpus into unigrams.  Then fit and transformed Partido Acción Nacional's tweets before obtaining the 10 most common unigrams.
cvec_uni = CountVectorizer(ngram_range=(1, 1), stop_words=spanish)

pan_uni_tweets = cvec_uni.fit_transform(pan_df['tweet'])

pan_uni_tweets_df = pd.DataFrame(data=pan_uni_tweets.A, columns=cvec_uni.get_feature_names_out())

pan_most_common_unigrams = pan_uni_tweets_df.sum().sort_values(ascending=False).head(10)
pan_most_common_unigrams

/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['actualmente', 'acuerdo', 'adelante', 'ademas', 'además', 'adrede', 'afirmó', 'agregó', 'ahi', 'ahora', 'ahí', 'al', 'algo', 'alguna', 'algunas', 'alguno', 'algunos', 'algún', 'alli', 'allí', 'alrededor', 'ambos', 'ampleamos', 'antano', 'antaño', 'ante', 'anterior', 'antes', 'apenas', 'aproximadamente', 'aquel', 'aquella', 'aquellas', 'aquello', 'aquellos', 'aqui', 'aquél', 'aquélla', 'aquéllas', 'aquéllos', 'aquí', 'arriba', 'arribaabajo', 'aseguró', 'asi', 'así', 'atras', 'aun', 'aunque', 'ayer', 'añadió', 'aún', 'bajo', 'bastante', 'bien', 'breve', 'buen', 'buena', 'buenas', 'bueno', 'buenos', 'cada', 'casi', 'cerca', 'cierta', 'ciertas', 'cierto', 'ciertos', 'cinco', 'claro', 'co', 'comentó', 'como', 'con', 'conmigo', 'conocer', 'conseguimos', 'conseguir', 'considera', 'consideró', 'c

de       4388
la       3062
el       2104
en       1793
rt       1792
https    1554
que      1550
co       1539
las      1391
para     1319
dtype: int64

In [78]:
# Fit and transformed the CountVectorizer object above on Partido Revolucionario Institucional's tweets before obtaining the 10 most common unigrams.

pri_uni_tweets = cvec_uni.fit_transform(pri_df['tweet'])

pri_uni_tweets_df = pd.DataFrame(data=pri_uni_tweets.A, columns=cvec_uni.get_feature_names_out())

pri_most_common_unigrams = pri_uni_tweets_df.sum().sort_values(ascending=False).head(10)
pri_most_common_unigrams

de       4819
la       3035
https    2282
co       2273
el       2226
que      2204
en       2203
con      1660
los      1582
las      1441
dtype: int64

In [79]:
# Fit and transformed the CountVectorizer object above on Partido de la Revolución Democrática's tweets before obtaining the 10 most common unigrams.

prd_uni_tweets = cvec_uni.fit_transform(prd_df['tweet'])

prd_uni_tweets_df = pd.DataFrame(data=prd_uni_tweets.A, columns=cvec_uni.get_feature_names_out())

prd_most_common_unigrams = prd_uni_tweets_df.sum().sort_values(ascending=False).head(10)
prd_most_common_unigrams

de       3736
la       3030
el       2216
rt       1870
en       1697
https    1594
co       1568
que      1417
las      1335
los       988
dtype: int64

In [80]:
# Fit and transformed the CountVectorizer object above on Partido Verde Ecologista de México's tweets before obtaining the 10 most common unigrams.

pv_uni_tweets = cvec_uni.fit_transform(pv_df['tweet'])

pv_uni_tweets_df = pd.DataFrame(data=pv_uni_tweets.A, columns=cvec_uni.get_feature_names_out())

pv_most_common_unigrams = pv_uni_tweets_df.sum().sort_values(ascending=False).head(10)
pv_most_common_unigrams

de       4456
https    3096
co       3094
la       2743
el       2103
en       1903
que      1312
los      1024
para     1005
por       968
dtype: int64

In [81]:
# Fit and transformed the CountVectorizer object above on Movimiento Ciudadano's tweets before obtaining the 10 most common unigrams.

mc_uni_tweets = cvec_uni.fit_transform(mc_df['tweet'])

mc_uni_tweets_df = pd.DataFrame(data=mc_uni_tweets.A, columns=cvec_uni.get_feature_names_out())

mc_most_common_unigrams = mc_uni_tweets_df.sum().sort_values(ascending=False).head(10)
mc_most_common_unigrams

de       2984
https    2831
co       2829
la       2455
que      1890
el       1836
en       1686
para     1113
las      1054
con       999
dtype: int64

In [82]:
# Fit and transformed the CountVectorizer object above on Morena's tweets before obtaining the 10 most common unigrams.

mrn_uni_tweets = cvec_uni.fit_transform(mrn_df['tweet'])

mrn_uni_tweets_df = pd.DataFrame(data=mrn_uni_tweets.A, columns=cvec_uni.get_feature_names_out())

mrn_most_common_unigrams = mrn_uni_tweets_df.sum().sort_values(ascending=False).head(10)
mrn_most_common_unigrams

de       4640
la       3693
https    3017
co       3005
el       2488
en       1562
que      1387
los      1087
del       913
es        874
dtype: int64

### 10 Most Common Bigrams

In [83]:
# Instantiating a CountVectorizer object that splits the documents in the corpus into bigrams.  Then fit and transformed Partido Acción Nacional's tweets before obtaining the 10 most common bigrams.
cvec_bi = CountVectorizer(ngram_range=(2, 2), stop_words=spanish)

pan_bi_tweets = cvec_bi.fit_transform(pan_df['tweet'])

pan_bi_tweets_df = pd.DataFrame(data=pan_bi_tweets.A, columns=cvec_bi.get_feature_names_out())

pan_most_common_bigrams = pan_bi_tweets_df.sum().sort_values(ascending=False).head(10)
pan_most_common_bigrams

https co           1535
de la               493
las los             314
de las              310
las familias        277
en el               247
en la               200
de junio            187
acción nacional     162
de los              144
dtype: int64

In [84]:
# Fit and transformed the CountVectorizer object above on Partido Revolucionario Institucional's tweets before obtaining the 10 most common bigrams.

pri_bi_tweets = cvec_bi.fit_transform(pri_df['tweet'])

pri_bi_tweets_df = pd.DataFrame(data=pri_bi_tweets.A, columns=cvec_bi.get_feature_names_out())

pri_most_common_bigrams = pri_bi_tweets_df.sum().sort_values(ascending=False).head(10)
pri_most_common_bigrams

https co                   2268
de la                       584
en el                       473
nuestro presidente          398
presidente alitomorenoc     376
el pri                      330
rt alitomorenoc             319
de las                      291
de junio                    286
para que                    285
dtype: int64

In [85]:
# Fit and transformed the CountVectorizer object above on Partido de la Revolución Democrática's tweets before obtaining the 10 most common bigrams.

prd_bi_tweets = cvec_bi.fit_transform(prd_df['tweet'])

prd_bi_tweets_df = pd.DataFrame(data=prd_bi_tweets.A, columns=cvec_bi.get_feature_names_out())

prd_most_common_bigrams = prd_bi_tweets_df.sum().sort_values(ascending=False).head(10)
prd_most_common_bigrams

https co              1557
rt prdmexico           644
de la                  596
en el                  369
de las                 332
el prd                 284
rt jesus_zambranog     278
las los                211
en la                  161
las mujeres            143
dtype: int64

In [86]:
# Fit and transformed the CountVectorizer object above on Partido Verde Ecologista de México's tweets before obtaining the 10 most common bigrams.

pv_bi_tweets = cvec_bi.fit_transform(pv_df['tweet'])

pv_bi_tweets_df = pd.DataFrame(data=pv_bi_tweets.A, columns=cvec_bi.get_feature_names_out())

pv_most_common_bigrams = pv_bi_tweets_df.sum().sort_values(ascending=False).head(10)
pv_most_common_bigrams

https co              3088
de la                  530
en el                  435
por un                 259
medio ambiente         243
de los                 235
en la                  220
de las                 146
rt karencastrejont     138
méxicounido https      138
dtype: int64

In [87]:
# Fit and transformed the CountVectorizer object above on Movimiento Ciudadano's tweets before obtaining the 10 most common bigrams.

mc_bi_tweets = cvec_bi.fit_transform(mc_df['tweet'])

mc_bi_tweets_df = pd.DataFrame(data=mc_bi_tweets.A, columns=cvec_bi.get_feature_names_out())

mc_most_common_bigrams = mc_bi_tweets_df.sum().sort_values(ascending=False).head(10)
mc_most_common_bigrams

https co                     2825
de la                         338
en el                         241
de las                        229
elfuturoesnaranja https       195
las los                       179
delríopresopolítico https     172
la bancadanaranja             152
en la                         149
que se                        140
dtype: int64

In [88]:
# Fit and transformed the CountVectorizer object above on Morena's tweets before obtaining the 10 most common bigrams.

mrn_bi_tweets = cvec_bi.fit_transform(mrn_df['tweet'])

mrn_bi_tweets_df = pd.DataFrame(data=mrn_bi_tweets.A, columns=cvec_bi.get_feature_names_out())

mrn_most_common_bigrams = mrn_bi_tweets_df.sum().sort_values(ascending=False).head(10)
mrn_most_common_bigrams

https co                       3005
de la                           879
de méxico                       563
conferenciapresidente https     446
el pueblo                       414
la 4t                           404
presidente de                   370
lopezobrador_ presidente        335
22 conferenciapresidente        255
las los                         250
dtype: int64

### 10 Most Common Trigrams

In [89]:
# Instantiating a CountVectorizer object that splits the documents in the corpus into trigrams.  Then fit and transformed Partido Acción Nacional's tweets before obtaining the 10 most common trigrams.
cvec_tri = CountVectorizer(ngram_range=(3, 3), stop_words=spanish)

pan_tri_tweets = cvec_tri.fit_transform(pan_df['tweet'])

pan_tri_tweets_df = pd.DataFrame(data=pan_tri_tweets.A, columns=cvec_tri.get_feature_names_out())

pan_most_common_trigrams = pan_tri_tweets_df.sum().sort_values(ascending=False).head(10)
pan_most_common_trigrams

votapan https co          128
de las familias           110
este de junio              88
para las familias          70
las los mexicanos          67
de las los                 66
las familias mexicanas     62
las los diputadospan       57
vamos con todo             56
de acción nacional         54
dtype: int64

In [90]:
# Fit and transformed the CountVectorizer object above on Partido Revolucionario Institucional's tweets before obtaining the 10 most common trigrams.

pri_tri_tweets = cvec_tri.fit_transform(pri_df['tweet'])

pri_tri_tweets_df = pd.DataFrame(data=pri_tri_tweets.A, columns=cvec_tri.get_feature_names_out())

pri_most_common_trigrams = pri_tri_tweets_df.sum().sort_values(ascending=False).head(10)
pri_most_common_trigrams

nuestro presidente alitomorenoc    374
este de junio                      270
votapri https co                   213
en el pri                          191
presidente alitomorenoc se         112
de nuestro partido                  95
las familias de                     79
alitomorenoc se encuentra           72
se encuentra en                     71
méxico https co                     69
dtype: int64

In [91]:
# Fit and transformed the CountVectorizer object above on Partido de la Revolución Democrática's tweets before obtaining the 10 most common trigrams.

prd_tri_tweets = cvec_tri.fit_transform(prd_df['tweet'])

prd_tri_tweets_df = pd.DataFrame(data=prd_tri_tweets.A, columns=cvec_tri.get_feature_names_out())

prd_most_common_trigrams = prd_tri_tweets_df.sum().sort_values(ascending=False).head(10)
prd_most_common_trigrams

en el prd             113
somosprd https co      99
votaprd https co       97
rt prdmexico el        87
rt prdmexico la        69
rt prdmexico en        68
en contra de           65
de las los             62
boletín https co       61
link boletín https     59
dtype: int64

In [92]:
# Fit and transformed the CountVectorizer object above on Partido Verde Ecologista de México's tweets before obtaining the 10 most common trigrams.

pv_tri_tweets = cvec_tri.fit_transform(pv_df['tweet'])

pv_tri_tweets_df = pd.DataFrame(data=pv_tri_tweets.A, columns=cvec_tri.get_feature_names_out())

pv_most_common_trigrams = pv_tri_tweets_df.sum().sort_values(ascending=False).head(10)
pv_most_common_trigrams

méxicounido https co     138
por un méxicoverde       114
méxicoverde https co     109
un méxicoverde https      74
aquí https co             73
actitudverde https co     72
el uso de                 71
yosoyverde https co       64
en el país                64
enhorabuena https co      60
dtype: int64

In [93]:
# Fit and transformed the CountVectorizer object above on Movimiento Ciudadano's tweets before obtaining the 10 most common trigrams.

mc_tri_tweets = cvec_tri.fit_transform(mc_df['tweet'])

mc_tri_tweets_df = pd.DataFrame(data=mc_tri_tweets.A, columns=cvec_tri.get_feature_names_out())

mc_most_common_trigrams = mc_tri_tweets_df.sum().sort_values(ascending=False).head(10)
mc_most_common_trigrams

elfuturoesnaranja https co      195
delríopresopolítico https co    172
google https co                 102
amazon https co                 102
apple https co                  102
spotify https co                 93
un nuevo trato                   93
tu plataforma favorita           80
en tu plataforma                 80
el medio ambiente                74
dtype: int64

In [94]:
# Fit and transformed the CountVectorizer object above on Morena's tweets before obtaining the 10 most common trigrams.

mrn_tri_tweets = cvec_tri.fit_transform(mrn_df['tweet'])

mrn_tri_tweets_df = pd.DataFrame(data=mrn_tri_tweets.A, columns=cvec_tri.get_feature_names_out())

mrn_most_common_trigrams = mrn_tri_tweets_df.sum().sort_values(ascending=False).head(10)
mrn_most_common_trigrams

conferenciapresidente https co    446
presidente de méxico              343
lopezobrador_ presidente de       335
22 conferenciapresidente https    250
francamente https co              192
para el pueblo                    172
la 4t es                          160
circulaenredes https co           132
de la conferenciapresidente       127
el presidente lopezobrador_       122
dtype: int64

### 10 Most Common 4-grams

In [95]:
# Instantiating a CountVectorizer object that splits the documents in the corpus into 4-grams.  Then fit and transformed Partido Acción Nacional's tweets before obtaining the 10 most common 4-grams.
cvec_quad = CountVectorizer(ngram_range=(4, 4), stop_words=spanish)

pan_quad_tweets = cvec_quad.fit_transform(pan_df['tweet'])

pan_quad_tweets_df = pd.DataFrame(data=pan_quad_tweets.A, columns=cvec_quad.get_feature_names_out())

pan_most_common_quadgrams = pan_quad_tweets_df.sum().sort_values(ascending=False).head(10)
pan_most_common_quadgrams

en paz https co                  50
descanse en paz https            50
el próximo de junio              48
la cámara de diputados           38
en la cámara de                  35
de las familias mexicanas        31
junio votapan https co           31
sus familiares seres queridos    30
de junio votapan https           29
de las los mexicanos             28
dtype: int64

In [96]:
# Fit and transformed the CountVectorizer object above on Partido Revolucionario Institucional's tweets before obtaining the 10 most common 4-grams.

pri_quad_tweets = cvec_quad.fit_transform(pri_df['tweet'])

pri_quad_tweets_df = pd.DataFrame(data=pri_quad_tweets.A, columns=cvec_quad.get_feature_names_out())

pri_most_common_quadgrams = pri_quad_tweets_df.sum().sort_values(ascending=False).head(10)
pri_most_common_quadgrams

nuestro presidente alitomorenoc se      112
presidente alitomorenoc se encuentra     69
alitomorenoc se encuentra en             68
este de junio votapri                    51
la cámara de diputados                   51
ganar este de junio                      47
este de junio vamos                      46
junio votapri https co                   41
de junio votapri https                   41
de las los mexicanos                     40
dtype: int64

In [97]:
# Fit and transformed the CountVectorizer object above on Partido de la Revolución Democrática's tweets before obtaining the 10 most common 4-grams.

prd_quad_tweets = cvec_quad.fit_transform(prd_df['tweet'])

prd_quad_tweets_df = pd.DataFrame(data=prd_quad_tweets.A, columns=cvec_quad.get_feature_names_out())

prd_most_common_quadgrams = prd_quad_tweets_df.sum().sort_values(ascending=False).head(10)
prd_most_common_quadgrams

link boletín https co                 58
el próximo de junio                   52
rt prdmexico en el                    39
jesus_zambranog link boletín https    35
millones de estudiantes los           34
de estudiantes los que                33
que les niega una                     33
los que les niega                     33
delfina gómez dé la                   33
dé la cara los                        33
dtype: int64

In [98]:
# Fit and transformed the CountVectorizer object above on Partido Verde Ecologista de México's tweets before obtaining the 10 most common 4-grams.

pv_quad_tweets = cvec_quad.fit_transform(pv_df['tweet'])

pv_quad_tweets_df = pd.DataFrame(data=pv_quad_tweets.A, columns=cvec_quad.get_feature_names_out())

pv_most_common_quadgrams = pv_quad_tweets_df.sum().sort_values(ascending=False).head(10)
pv_most_common_quadgrams

un méxicoverde https co           74
por un méxicoverde https          68
sen raulbccue https co            53
motivo de su cumpleaños           42
dip arturoesc https co            42
con motivo de su                  42
medio ambiente https co           38
enviamos una gran felicitación    37
dip betymanrique https co         36
de su cumpleaños sigamos          35
dtype: int64

In [99]:
# Fit and transformed the CountVectorizer object above on Movimiento Ciudadano's tweets before obtaining the 10 most common 4-grams.

mc_quad_tweets = cvec_quad.fit_transform(mc_df['tweet'])

mc_quad_tweets_df = pd.DataFrame(data=mc_quad_tweets.A, columns=cvec_quad.get_feature_names_out())

mc_most_common_quadgrams = mc_quad_tweets_df.sum().sort_values(ascending=False).head(10)
mc_most_common_quadgrams

en tu plataforma favorita            80
escúchanos en tu plataforma          63
tu plataforma favorita spotify       55
plataforma favorita spotify https    55
favorita spotify https co            55
un nuevo trato por                   46
por el medio ambiente                39
en redes https co                    30
medio ambiente https co              30
circula en redes https               28
dtype: int64

In [100]:
# Fit and transformed the CountVectorizer object above on Morena's tweets before obtaining the 10 most common 4-grams.

mrn_quad_tweets = cvec_quad.fit_transform(mrn_df['tweet'])

mrn_quad_tweets_df = pd.DataFrame(data=mrn_quad_tweets.A, columns=cvec_quad.get_feature_names_out())

mrn_most_common_quadgrams = mrn_quad_tweets_df.sum().sort_values(ascending=False).head(10)
mrn_most_common_quadgrams

lopezobrador_ presidente de méxico     335
22 conferenciapresidente https co      250
compartimos el resumen de              116
te compartimos el resumen              116
resumen de la conferenciapresidente    116
el resumen de la                       116
la conferenciapresidente del día       115
4t es información para                 115
conferenciapresidente del día de       115
de la conferenciapresidente del        115
dtype: int64

### 10 Most Common 5-grams

In [101]:
# Instantiating a CountVectorizer object that splits the documents in the corpus into 5-grams.  Then fit and transformed Partido Acción Nacional's tweets before obtaining the 10 most common 5-grams.
cvec_quint = CountVectorizer(ngram_range=(5, 5), stop_words=spanish)

pan_quint_tweets = cvec_quint.fit_transform(pan_df['tweet'])

pan_quint_tweets_df = pd.DataFrame(data=pan_quint_tweets.A, columns=cvec_quint.get_feature_names_out())

pan_most_common_quintgrams = pan_quint_tweets_df.sum().sort_values(ascending=False).head(10)
pan_most_common_quintgrams

descanse en paz https co                        50
en la cámara de diputados                       33
de junio votapan https co                       29
seres queridos descanse en paz                  25
para todas todos https co                       23
lamentamos profundamente el fallecimiento de    23
queridos descanse en paz https                  23
de esta semana https co                         22
familiares seres queridos descanse en           21
sus familiares seres queridos descanse          21
dtype: int64

In [102]:
# Fit and transformed the CountVectorizer object above on Partido Revolucionario Institucional's tweets before obtaining the 10 most common 5-grams.

pri_quint_tweets = cvec_quint.fit_transform(pri_df['tweet'])

pri_quint_tweets_df = pd.DataFrame(data=pri_quint_tweets.A, columns=cvec_quint.get_feature_names_out())

pri_most_common_quintgrams = pri_quint_tweets_df.sum().sort_values(ascending=False).head(10)
pri_most_common_quintgrams

nuestro presidente alitomorenoc se encuentra     69
presidente alitomorenoc se encuentra en          65
de junio votapri https co                        41
este de junio votapri https                      41
este de junio https co                           38
nuestro presidente alitomorenoc se reúne         34
vamos ganar este de junio                        34
este de junio vamos ganar                        30
nuestro presidente alitomorenoc tomó protesta    29
presidente alitomorenoc se reúne con             27
dtype: int64

In [103]:
# Fit and transformed the CountVectorizer object above on Partido de la Revolución Democrática's tweets before obtaining the 10 most common 5-grams.

prd_quint_tweets = cvec_quint.fit_transform(prd_df['tweet'])

prd_quint_tweets_df = pd.DataFrame(data=prd_quint_tweets.A, columns=cvec_quint.get_feature_names_out())

prd_most_common_quintgrams = prd_quint_tweets_df.sum().sort_values(ascending=False).head(10)
prd_most_common_quintgrams

jesus_zambranog link boletín https co    35
cara los millones de estudiantes         33
de estudiantes los que les               33
gómez dé la cara los                     33
los millones de estudiantes los          33
millones de estudiantes los que          33
les niega una educación de               33
delfina gómez dé la cara                 33
que les niega una educación              33
la cara los millones de                  33
dtype: int64

In [104]:
# Fit and transformed the CountVectorizer object above on Partido Verde Ecologista de México's tweets before obtaining the 10 most common 5-grams.

pv_quint_tweets = cvec_quint.fit_transform(pv_df['tweet'])

pv_quint_tweets_df = pd.DataFrame(data=pv_quint_tweets.A, columns=cvec_quint.get_feature_names_out())

pv_most_common_quintgrams = pv_quint_tweets_df.sum().sort_values(ascending=False).head(10)
pv_most_common_quintgrams

por un méxicoverde https co             68
con motivo de su cumpleaños             42
motivo de su cumpleaños sigamos         35
su cumpleaños sigamos trabajando por    32
enviamos una gran felicitación un       32
cumpleaños sigamos trabajando por un    32
de su cumpleaños sigamos trabajando     32
gran felicitación un fuerte abrazo      31
una gran felicitación un fuerte         31
unidos por un méxicoverde https         30
dtype: int64

In [105]:
# Fit and transformed the CountVectorizer object above on Movimiento Ciudadano's tweets before obtaining the 10 most common 5-grams.

mc_quint_tweets = cvec_quint.fit_transform(mc_df['tweet'])

mc_quint_tweets_df = pd.DataFrame(data=mc_quint_tweets.A, columns=cvec_quint.get_feature_names_out())

mc_most_common_quintgrams = mc_quint_tweets_df.sum().sort_values(ascending=False).head(10)
mc_most_common_quintgrams

escúchanos en tu plataforma favorita        63
tu plataforma favorita spotify https        55
plataforma favorita spotify https co        55
en tu plataforma favorita spotify           55
circula en redes https co                   28
nuevo trato por el medio                    23
andamosbienpuestxs para regular https co    23
trato por el medio ambiente                 23
un nuevo trato por el                       22
un nuevo trato al medio                     21
dtype: int64

In [106]:
# Fit and transformed the CountVectorizer object above on Morena's tweets before obtaining the 10 most common 5-grams.

mrn_quint_tweets = cvec_quint.fit_transform(mrn_df['tweet'])

mrn_quint_tweets_df = pd.DataFrame(data=mrn_quint_tweets.A, columns=cvec_quint.get_feature_names_out())

mrn_most_common_quintgrams = mrn_quint_tweets_df.sum().sort_values(ascending=False).head(10)
mrn_most_common_quintgrams

compartimos el resumen de la               116
el resumen de la conferenciapresidente     116
te compartimos el resumen de               116
la 4t es información para                  115
la conferenciapresidente del día de        115
conferenciapresidente del día de hoy       115
de la conferenciapresidente del día        115
resumen de la conferenciapresidente del    114
es información para el pueblo              113
para el pueblo democracia transparencia    113
dtype: int64